In [1]:
# !pip3 install flask
# !pip3 install line-bot-sdk

#上傳至heroku 須把相關安裝套件註解

In [2]:
'''

整體功能描述

應用伺服器主結構樣板
    用來定義使用者傳送消息時，應該傳到哪些方法上
        比如收到文字消息時，轉送到文字專屬處理方法

消息專屬方法定義
    當收到文字消息，從資料夾內提取消息，並進行回傳。
    
啟動應用伺服器    

'''

'\n\n整體功能描述\n\n應用伺服器主結構樣板\n    用來定義使用者傳送消息時，應該傳到哪些方法上\n        比如收到文字消息時，轉送到文字專屬處理方法\n\n消息專屬方法定義\n    當收到文字消息，從資料夾內提取消息，並進行回傳。\n    \n啟動應用伺服器    \n\n'

In [3]:
'''

Application 主架構

'''

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("./line_secret_key",'r',encoding='utf8'))

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'



#ok

In [4]:
'''

消息判斷器

讀取指定的json檔案後，把json解析成不同格式的SendMessage

讀取檔案，
把內容轉換成json
將json轉換成消息
放回array中，並把array傳出。

'''

# 引用會用到的套件
from linebot.models import (
    ImagemapSendMessage,TextSendMessage,ImageSendMessage,LocationSendMessage,FlexSendMessage,VideoSendMessage
)

from linebot.models.template import (
    ButtonsTemplate,CarouselTemplate,ConfirmTemplate,ImageCarouselTemplate
    
)

from linebot.models.template import *

def detect_json_array_to_new_message_array(fileName):
    
    #開啟檔案，轉成json
    with open(fileName,'r',encoding='utf8') as f:
        jsonArray = json.load(f)
    
    # 解析json
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))    


    # 回傳
    return returnArray


#ok

In [5]:
'''

handler處理關注消息

用戶關注時，讀取 素材 -> 關注 -> reply.json

將其轉換成可寄發的消息，傳回給Line

'''

# 引用套件
from linebot.models import (
    FollowEvent
)

# 關注事件處理
@handler.add(FollowEvent)
def process_follow_event(event):
    
    # 讀取並轉換
    result_message_array =[]
    replyJsonPath = "material/follow/reply.json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

    # 消息發送
    line_bot_api.reply_message(
        event.reply_token,
        result_message_array
    )

    # 從follow資料夾內，取出圖文選單id，並告知line，綁定用戶
    linkRichMenuId = open("material/follow/rich_menu_id", 'r').read()
    line_bot_api.link_rich_menu_to_user(event.source.user_id, linkRichMenuId)
    

    
    
#ok

In [6]:
'''

handler處理Postback Event

載入功能選單與啟動特殊功能

解析postback的data，並按照data欄位判斷處理

現有三個欄位
menu, folder, tag

若folder欄位有值，則
    讀取其reply.json，轉譯成消息，並發送

若menu欄位有值，則
    讀取其rich_menu_id，並取得用戶id，將用戶與選單綁定
    讀取其reply.json，轉譯成消息，並發送

'''
from linebot.models import (
    PostbackEvent
)

from urllib.parse import parse_qs 

@handler.add(PostbackEvent)
def process_postback_event(event):
    
    query_string_dict = parse_qs(event.postback.data)
    
    print(query_string_dict)
    if 'folder' in query_string_dict:
    
        result_message_array =[]

        replyJsonPath = 'material/'+query_string_dict.get('folder')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    elif 'menu' in query_string_dict:
 
        linkRichMenuId = open("material/"+query_string_dict.get('menu')[0]+'/rich_menu_id', 'r').read()
        line_bot_api.link_rich_menu_to_user(event.source.user_id,linkRichMenuId)
        
        replyJsonPath = 'material/'+query_string_dict.get('menu')[0]+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)
  
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )
    
    
#ok

In [7]:
'''

handler處理文字消息

收到用戶回應的文字消息，
按文字消息內容，往素材資料夾中，找尋以該內容命名的資料夾，讀取裡面的reply.json

轉譯json後，將消息回傳給用戶

'''

# 引用套件
from linebot.models import (
    MessageEvent, TextMessage
)

# 文字消息處理
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(event):
    global saveAns
    
    if event.message.text[0] == '@':
        print(saveAns, 'text process')
        
        
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=saveAns))
        
    elif event.message.text[0] == '#':
        line_bot_api.reply_message(
            event.reply_token, 
            TextSendMessage(text="模型處理中~請稍後~"))
        
        
        
        txt = event.message.text[1: ]
        tg, tp = txt.split(' ')[0], txt.split(' ')[1]

        gCodeTnameCleanCsv = r"./g_code_tname_clean.csv"   #跟改資料路徑
        ans, x, y_df = tiw.find_same_word(tg, tp, gCodeTnameCleanCsv)
        line_bot_api.reply_message(
            event.reply_token, 
            TextSendMessage(text="hello"))

            

#         print(ans)


        for i in range(len(ans.index)):
            saveAns +=ans['name : '].iloc[i] + str(ans['score : '].iloc[i]) + '\n'
#             print(saveAns)
            
            

            
            
        print('可以看結果')
    else:
        # 讀取本地檔案，並轉譯成消息
        result_message_array =[]
        replyJsonPath = "material/"+event.message.text+"/reply.json"
        result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

        # 發送
        line_bot_api.reply_message(
            event.reply_token,
            result_message_array
        )

In [8]:
'''

若收到圖片消息時，

先回覆用戶文字消息，並從Line上將照片拿回。

'''
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage,ImageMessage
)


import tmark_input_word as tiw

saveAns = ''

@handler.add(MessageEvent, message=ImageMessage)
def handle_message(event):
    global saveAns
    
    line_bot_api.reply_message(
        event.reply_token, 
        TextSendMessage(text='照片已上傳成功~' + ' \n' + '請輸入 #來源,類別（ex. #生命 032）'))
    
#     line_bot_api.reply_message(
#         event.reply_token,
#         TextSendMessage(text='stringAns'))    
 
#     message_content = line_bot_api.get_message_content(event.message.id)

    
#     with open('../images/'+event.message.id+'.jpg', 'wb') as fd:
#         for chunk in message_content.iter_content():
#             fd.write(chunk)


In [ ]:
'''

Application 運行（開發版）

'''
if __name__ == "__main__":
    app.run(host='0.0.0.0')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jan/2021 16:55:49] "POST / HTTP/1.1" 200 -
c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\linebot\webhook.py:260: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  self.__invoke_func(func, event, payload)
[2021-01-17 16:56:47,440] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\_compat.py", line 39

{}


[2021-01-18 10:27:46,282] ERROR in app: Exception on / [POST]
Traceback (most recent call last):
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "c:\users\uesr01\pycharmprojects\商標專題\venv\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**re

In [ ]:
'''

Application 運行（heroku版）

'''

# import os
# if __name__ == "__main__":
#     app.run(host='0.0.0.0',port=os.environ['PORT'])